In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, f1_score
from imblearn.over_sampling import SMOTE
from rdkit import Chem
from rdkit.Chem import Descriptors
import joblib
import json

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
# === Load datasets ===
chembl = pd.read_csv("chembl.csv")  # Columns: SMILES, Name
aqsol = pd.read_csv("curated-solubility-dataset.csv")  # Columns include InChIKey or Name, not always SMILES
tox21 = pd.read_csv("ALLPCPPREDS_080516.csv")  # Tox21: indexed by CAS or ID, not direct SMILES


In [ ]:
# === Canonical SMILES from ChEMBL for merging ===
chembl['Canonical_SMILES'] = chembl['SMILES'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x)) if Chem.MolFromSmiles(x) else None)


In [ ]:
# === Merge carefully based on compound Name (ChEMBL) and identifier column from other datasets ===
# You may need to map Name ↔ InChIKey or external DB if SMILES not present in Tox21/AqSolDB
merged = chembl.merge(aqsol, left_on='Name', right_on='Name', how='inner')
merged = merged.merge(tox21, left_on='Name', right_on='Name', how='inner')


In [ ]:
# === Feature Extraction (RDKit Descriptors) ===
def extract_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [0]*8
    return [
        Descriptors.MolWt(mol),
        Descriptors.MolLogP(mol),
        Descriptors.TPSA(mol),
        Descriptors.NumHDonors(mol),
        Descriptors.NumHAcceptors(mol),
        Descriptors.RingCount(mol),
        Descriptors.NumRotatableBonds(mol),
        Descriptors.FractionCSP3(mol)
    ]

In [ ]:
merged[['MolWt','MolLogP','TPSA','HDonors','HAcceptors','Rings','RotBonds','CSP3']] = merged['Canonical_SMILES'].apply(lambda x: pd.Series(extract_descriptors(x)))

# === Prepare Input/Output ===
X = merged[['MolWt','MolLogP','TPSA','HDonors','HAcceptors','Rings','RotBonds','CSP3']]
y = merged['toxicity_label'] if 'toxicity_label' in merged.columns else merged.iloc[:, 6]  # Fallback to a toxicity column

# === Normalize Features ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# === SMOTE for balancing ===
sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_scaled, y)


In [ ]:
# === K-Fold Cross-Validation ===
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

all_metrics = []
final_model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=300,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.9,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)


In [ ]:
for fold, (train_idx, test_idx) in enumerate(skf.split(X_resampled, y_resampled)):
    X_train, X_test = X_resampled[train_idx], X_resampled[test_idx]
    y_train, y_test = y_resampled[train_idx], y_resampled[test_idx]

    final_model.fit(X_train, y_train)
    y_pred = final_model.predict(X_test)
    y_prob = final_model.predict_proba(X_test)[:, 1]

    metrics = {
        'Fold': fold+1,
        'Accuracy': accuracy_score(y_test, y_pred),
        'F1': f1_score(y_test, y_pred, average='macro'),
        'ROC_AUC': roc_auc_score(y_test, y_prob)
    }
    all_metrics.append(metrics)

# === Save Model (Final Save After K-Fold Completion) ===
joblib.dump(final_model, "drug_toxicity_model.pkl")
joblib.dump(scaler, "scaler.pkl")
print("Model and scaler saved successfully after training.")


In [ ]:
def predict_and_format(smiles, compound_name=""):
    desc = extract_descriptors(smiles)
    scaled = scaler.transform([desc])
    prob = final_model.predict_proba(scaled)[0]
    pred = final_model.predict(scaled)[0]

    result = {
        "Name": compound_name,
        "SMILES": smiles,
        "Predicted Properties": {
            "MolWt": desc[0],
            "MolLogP": desc[1],
            "TPSA": desc[2],
            "HDonors": desc[3],
            "HAcceptors": desc[4],
            "RingCount": desc[5],
            "Rotatable Bonds": desc[6],
            "FractionCSP3": desc[7]
        },
        "Toxicity Prediction": int(pred),
        "Toxicity Confidence": float(prob[1])
    }
    return json.dumps(result, indent=2)


In [ ]:
print(predict_and_format("c1(c2nc(N=C(N)N)sc2)cn(c(c1)C)C", "CHEMBL153534"))
